In [1]:
import torch
import numpy as np 
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Subset
from torchvision.datasets import ImageFolder

## Alexnet Model 

* get model architecture:

In [2]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
model.eval()

Using cache found in /Users/duuta/.cache/torch/hub/pytorch_vision_v0.10.0
/Users/duuta/.pyenv/versions/3.9.16/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/duuta/.pyenv/versions/3.9.16/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

# define hook for intermediate activations

In [19]:
num_rois = 193600
num_stimulus = 2800
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

def ghook(model, input, output):
    return intermediate_activations.append(output)

intermediate_activations = []

arr00 = np.empty((num_stimulus, num_rois), dtype=float)


## Using subset of ImageNet data, define transforms on data

In [23]:
batch_size = 1

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

transform = transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean, std),
         
            ]
        )

imagenet_data = ImageFolder('/Users/duuta/ppp/notebooks/robustSigs/ImageNet_data/imagenet-mini/', transform=transform)
imagenet_data2800 = Subset(imagenet_data, np.arange(2800))
data_loader = torch.utils.data.DataLoader(imagenet_data2800, 
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=10) 

num_epochs = int(len(imagenet_data2800)/ batch_size)

In [ ]:
# features_list = [0, 3, 6, 8, 10]
# number of neurons 193600 
# set batch size and scale with 

In [24]:
sample_target_layer = model.features[0]
for epoch  in range(1000):
    for j, (input, _) in enumerate(iter(data_loader)):
        hook_handle = sample_target_layer.register_forward_hook(get_activation('feats'))
        _ = model(input)
        arr00[j, :] = activation['feats'].reshape(1, num_rois)

hook_handle.remove()

KeyboardInterrupt: 

* move imagenet data to nemo, computation cost is 1min/image 